In [ ]:
import base64
import requests
from bs4 import BeautifulSoup
import re

# Send a GET request to the URL
response = requests.get("https://www.nike.com/w/shoes-y7ok")

# Parse the content of the request with BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Find the first 5 product cards on the page
product_cards = soup.find_all("div", {"class": "product-card__body"}, limit=5)

# For each product card, extract the product name, link, image source, and price
for i, product_card in enumerate(product_cards):
    # Product name
    product_name = product_card.find("div", {"class": "product-card__title"}).text

    # Product link
    product_link = product_card.find("a", {"class": "product-card__link-overlay"})["href"]

    # Image source
    image_src = product_card.find("img", {"class": "product-card__hero-image"})["src"]

    if image_src.startswith("data:image"):
        # Remove data:image/png;base64, part
        image_base64 = re.sub('^data:image/.+;base64,', '', image_src)
        # Save image
        with open(f'product_{i}.png', 'wb') as f:
            f.write(base64.b64decode(image_base64))
    else:
        # Download the image
        urllib.request.urlretrieve(image_src, f"product_{i}.jpg")
        
    image_src = image_base64

    # Product price
    product_price = product_card.find("div", {"class": "product-price"}).text

    # Make a request to the individual product page
    product_response = requests.get(product_link)

    # Parse the content of the product page
    product_soup = BeautifulSoup(product_response.text, "html.parser")

    # Find the product description
    product_description = product_soup.find("div", {"class": "description-preview"}).text

    print(f"Product Name: {product_name}")
    print(f"Product Link: {product_link}")
    print(f"Image Link: {image_src}")
    print(f"Product Price: {product_price}")
    print(f"Product Description: {product_description}")
    print("\n-----\n")


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import csv
import requests

# Set up the Selenium driver (in this case, Chrome)
driver = webdriver.Chrome(ChromeDriverManager().install())

# Go to the webpage
driver.get("https://www.nike.com/w/shoes-y7ok")

# Find the first 5 product cards on the page
product_cards = driver.find_elements(By.CLASS_NAME, "product-card__body")[:100]

for i, product_card in enumerate(product_cards):
    # Product name
    product_name = product_card.find_element(By.CLASS_NAME, "product-card__title").text

    # Product link
    product_link = product_card.find_element(By.CLASS_NAME, "product-card__link-overlay").get_attribute('href')

    # Image source
    image_src = product_card.find_element(By.CLASS_NAME, "product-card__hero-image").get_attribute('src')

    # Product price
    product_price = product_card.find_element(By.CLASS_NAME, "product-price").text
    
    # Make a request to the individual product page
    product_response = requests.get(product_link)

    # Parse the content of the product page
    product_soup = BeautifulSoup(product_response.text, "html.parser")

    # Find the product description
    product_description = product_soup.find("div", {"class": "description-preview"}).text


    print(f"Product Name: {product_name}")
    print(f"Product Link: {product_link}")
    print(f"Image Link: {image_src}")
    print(f"Product Price: {product_price}")
    print(f"Product Description: {product_description}")
    print("\n-----\n")

# Close the driver
driver.quit()


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import requests

# Set up the Selenium driver (in this case, Chrome)
driver = webdriver.Chrome(ChromeDriverManager().install())

# Go to the webpage
driver.get("https://www.nike.com/w/shoes-y7ok")

# Open the CSV file in write mode
with open('nike_products.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["Product Name", "Product Link", "Image Link", "Product Price", "Product Description"])

    # Find the first 10 product cards on the page
    product_cards = driver.find_elements(By.CLASS_NAME, "product-card__body")[:500]

    for product_card in product_cards:
        # Product name
        product_name = product_card.find_element(By.CLASS_NAME, "product-card__title").text

        # Product link
        product_link = product_card.find_element(By.CLASS_NAME, "product-card__link-overlay").get_attribute('href')

        # Image source
        image_src = product_card.find_element(By.CLASS_NAME, "product-card__hero-image").get_attribute('src')

        # Product price
        product_price = product_card.find_element(By.CLASS_NAME, "product-price").text

        # Make a request to the individual product page
        product_response = requests.get(product_link)

        
        # Parse the content of the product page
        product_soup = BeautifulSoup(product_response.text, "html.parser")
        
        # Find the product description
        product_description_element = product_soup.find("div", {"class": "description-preview"})

        if product_description_element is None:
            product_description = ""
        else:
            product_description = product_description_element.text
        

        # Write the data to the CSV file
        writer.writerow([product_name, product_link, image_src, product_price, product_description])

# Close the driver
driver.quit()
